In [6]:
import cv2
import numpy as np

def swap_faces(source_image, destination_image):
    # Load the images
    source_img = cv2.imread(source_image)
    destination_img = cv2.imread(destination_image)

    # Load Haar cascades for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    # Convert the images to grayscale
    source_gray = cv2.cvtColor(source_img, cv2.COLOR_BGR2GRAY)
    destination_gray = cv2.cvtColor(destination_img, cv2.COLOR_BGR2GRAY)

    # Detect faces in both images
    source_faces = face_cascade.detectMultiScale(source_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    destination_faces = face_cascade.detectMultiScale(destination_gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(source_faces) == 0 or len(destination_faces) == 0:
        print("No faces detected in one or both images.")
        return None

    # Assume we are only swapping one face from each image
    source_x, source_y, source_w, source_h = source_faces[0]
    destination_x, destination_y, destination_w, destination_h = destination_faces[0]

    # Extract the regions of interest (ROI) around the faces
    source_roi = source_img[source_y:source_y + source_h, source_x:source_x + source_w]
    destination_roi = destination_img[destination_y:destination_y + destination_h, destination_x:destination_x + destination_w]

    # Define paths to the pre-trained facial landmark model (download from OpenCV's GitHub repository)
    model_path = "path/to/opencv_face_detector_uint8.pb"
    config_path = "path/to/opencv_face_detector.pbtxt"

    # Load the pre-trained facial landmark model
    net = cv2.dnn.readNetFromTensorflow(model_path, config_path)
    blob = cv2.dnn.blobFromImage(destination_roi, 1.0, (300, 300), [104, 117, 123], False)

    # Run the facial landmark model on the destination face ROI
    net.setInput(blob)
    landmarks = net.forward()

    # Extract the facial landmarks
    destination_points = []
    for i in range(68):
        x = int(landmarks[0, 0, i, 0] * destination_w) + destination_x
        y = int(landmarks[0, 0, i, 1] * destination_h) + destination_y
        destination_points.append((x, y))

    # Find a convex hull for the destination face landmarks
    hull_points = cv2.convexHull(np.array(destination_points), returnPoints=True)

    # Create a mask to extract the face region from the source image
    mask = np.zeros(source_img.shape[:2], dtype=np.uint8)
    cv2.fillPoly(mask, [np.array(destination_points)], 255)

    # Find the affine transformation matrix to align the source face to the destination face
    transformation_matrix, _ = cv2.estimateAffinePartial2D(np.array(destination_points), np.array(source_faces)[:,:2])

    # Apply the transformation to the source face
    source_face_warped = cv2.warpAffine(source_roi, transformation_matrix, (destination_w, destination_h))

    # Extract the face region from the destination image using the mask
    destination_face_masked = cv2.bitwise_and(destination_roi, destination_roi, mask=mask)

    # Combine the warped source face with the destination face
    swapped_face = cv2.add(destination_face_masked, source_face_warped)

    # Replace the face region in the destination image with the swapped face
    destination_img[destination_y:destination_y + destination_h, destination_x:destination_x + destination_w] = swapped_face

    return destination_img

if __name__ == "__main__":
    source_image_path = "data/P8/img1.png"
    destination_image_path = "data/P8/img2.png"

    swapped_image = swap_faces(source_image_path, destination_image_path)

    if swapped_image is not None:
        cv2.imshow("Swapped Image", swapped_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/dnn/src/caffe/caffe_io.cpp:1138: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "path/to/opencv_face_detector_uint8.pb" in function 'ReadProtoFromBinaryFile'
